# For Results in Realtime
Use Tweepy's Library. There are ways of storing this data continuously in a SQLite database, or locally on your computer. See https://towardsdatascience.com/mining-live-twitter-data-for-sentiment-analysis-of-events-d69aa2d136a1 for more info on contiunuous storage and sentiment analysis.

Below code is a modification of what was found on https://www.geeksforgeeks.org/twitter-sentiment-analysis-using-python/. This is basically a learning tool because, so far, the results are pretty shit at analysis. Suggestions moving forward: implement VADER as it is calibrated to social media data already (rather than movie reviews, like textblob). If you're feeling ambitious, get some preliminary data first, build your own damn model on what you're trying to predict, and then use that retroactively as well. 


STEPS: 
1. Build a continuous streamer to store data. Probably don't need to pay for storage if you just do it while your computer is running. Or, fuck it, pay for storage. 
2. Tag the data. This will be pretty damn painstaking. You'll really have to narrow down your tracker keywords, and make sure they get what you want, otherwise enjoy labelling MOUNTAINS of tweets that will be mostly noise. 
3. Train the model on that data. You better make sure it's predictive otherwise goodnight project.
4. Test the data on historical data to see if it is accurate. Hopefully whatever keywords you identify haven't changed much over time. 
5. Build a trendline of the data. Hopefully it matches all of the legislation.
6. Make forecasts for the rest. 


In [26]:


import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 

class TwitterClient(object): 
	''' 
	Generic Twitter Class for sentiment analysis. 
	'''
	def __init__(self): 
		''' 
		Class constructor or initialization method. 
		'''
		# keys and tokens from the Twitter Dev Console 
		consumer_key = 'qtIWsigy1FG4kyLIe8darX3vf'
		consumer_secret = 'fEt7blLkFFWMppZv2cMt58PN1o7ridztG8SzoNnM5XRR8hwgqb'
		access_token = '1507759020-g9om0JN1evu98wtSjnOwVRDK3sXLX0QdpzRohw5'
		access_token_secret = 'lrerVx5Y2uoxCRoHf7Xhfco2NSiw5T8YKojcXISTQz4ru'

		# attempt authentication 
		try: 
			# create OAuthHandler object 
			self.auth = OAuthHandler(consumer_key, consumer_secret) 
			# set access token and secret 
			self.auth.set_access_token(access_token, access_token_secret) 
			# create tweepy API object to fetch tweets 
			self.api = tweepy.API(self.auth) 
		except: 
			print("Error: Authentication Failed") 

	def clean_tweet(self, tweet): 
		''' 
		Utility function to clean tweet text by removing links, special characters 
		using simple regex statements. 
		'''
		return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 

	def get_tweet_sentiment(self, tweet): 
		''' 
		Utility function to classify sentiment of passed tweet 
		using textblob's sentiment method 
		'''
		# create TextBlob object of passed tweet text 
		analysis = TextBlob(self.clean_tweet(tweet)) 
		# set sentiment 
		if analysis.sentiment.polarity > 0: 
			return 'positive'
		elif analysis.sentiment.polarity == 0: 
			return 'neutral'
		else: 
			return 'negative'

	def get_tweets(self, query, count = 10): 
		''' 
		Main function to fetch tweets and parse them. 
		'''
		# empty list to store parsed tweets 
		tweets = [] 

		try: 
			# call twitter api to fetch tweets 
			fetched_tweets = self.api.search(q = query, count = count) 

			# parsing tweets one by one 
			for tweet in fetched_tweets: 
				# empty dictionary to store required params of a tweet 
				parsed_tweet = {} 

				# saving text of tweet 
				parsed_tweet['text'] = tweet.text 
				# saving sentiment of tweet 
				parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 

				# appending parsed tweet to tweets list 
				if tweet.retweet_count > 0: 
					# if tweet has retweets, ensure that it is appended only once 
					if parsed_tweet not in tweets: 
						tweets.append(parsed_tweet) 
				else: 
					tweets.append(parsed_tweet) 

			# return parsed tweets 
			return tweets 

		except tweepy.TweepError as e: 
			# print error (if any) 
			print("Error : " + str(e)) 


In [27]:

def main(): 
	# creating object of TwitterClient Class 
	api = TwitterClient() 
	# calling function to get tweets 
	tweets = api.get_tweets(query = 'open data', count = 200) 

	# picking positive tweets from tweets 
	ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
	# percentage of positive tweets 
	print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
	# picking negative tweets from tweets 
	ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
	# percentage of negative tweets 
	print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
	# percentage of neutral tweets 
	print("Neutral tweets percentage: {} %".format(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets)))
	print("\n\nPositive tweets:") 
	for tweet in ptweets[:10]: 
		print(tweet['text']) 

	# printing first 5 negative tweets 
	print("\n\nNegative tweets:") 
	for tweet in ntweets[:10]: 
		print(tweet['text']) 

if __name__ == "__main__": 
	# calling main function 
	main() 


Positive tweets percentage: 49.31506849315068 %
Negative tweets percentage: 6.8493150684931505 %
Neutral tweets percentage: 43.83561643835616 %


Positive tweets:
RT @thehowie: This is why testing/more data would have been so helpful. Risk of keeping public schools open is to elderly &amp; otherwise compr…
RT @CornellILR_News: Research co-authored by @CornnellILR asst. prof. @iajunwa details results of the first nationally representative surve…
Lockdown is open source, so you can see exactly what it's doing and *not* doing.
Never trust a closed source securi… https://t.co/TC9m2kK5dj
Spectator entrance, WIAA Boys Basketball, at Brillion High School, Xavier vs Wrightstown.
https://t.co/KFFhu3ubek
1… https://t.co/4cUQDSpzFb
@rowlsmanthorpe @drdrmc @lilianedwards Open Banking isn't about that kind of openness, it's about giving customers… https://t.co/M1z4Rv5zpa
I love seeing data that curation makes for more usable open data (as opposed to researcher self-deposit in a data repository). #R

# Streamer Data
Look below to see some sample code from this article:https://towardsdatascience.com/mining-live-twitter-data-for-sentiment-analysis-of-events-d69aa2d136a1

In [ ]:
import os
import json
import tweepy
from tweepy import Stream                   # Useful in Step 3
from tweepy.streaming import StreamListener # Useful in Step 3
consumer_key = os.getenv(“CONSUMER_KEY_TWITTER”)
consumer_secret = os.getenv(“CONSUMER_SECRET_TWITTER”)
access_token = os.getenv(“ACCESS_KEY_TWITTER”)
access_token_secret = os.getenv(“ACCESS_SECRET_TWITTER”)
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)


## Defining a class Listener so that you can manipulate data.
class listener(StreamListener):
    def on_data(self, data):
        data = json.loads(data)
        # Filter out non-English Tweets
        if data.get("lang") != "en": 
            return True
        try:
            timestamp = data['timestamp_ms']            
            # Get longer 280 char tweets if possible
            if data.get("extended_tweet"):
                tweet = data['extended_tweet']["full_text"]
            else:
                tweet = data["text"]
            url = "https://www.twitter.com/i/web/status/" +   
                   data["id_str"]
            user = data["user"]["screen_name"]
            verified = data["user"]["verified"]                  
            write_to_csv([timestamp, tweet, user, verified, url])
            ## include a write_to_csv statement to save this to your device. O/W setup eternal storage.
            
        except KeyError as e:
            print("Keyerror:", e)

        return True

    def on_error(self, status):
        print(status)
        
        
        
       
    
## Defining a function to actually stream and write. 
#Note that we use the extended mode to capture as much info as possible. 
def stream_and_write(table, track=None):
    try:
        twitterStream = Stream(auth, listener(), 
                               tweet_mode='extended')
        twitterStream.filter(track=["AAPL", "AMZN", "UBER"])
    except Exception as e:
        print("Error:", str(e))
        time.sleep(5)
        
        
## Here's the vader model. 
#You can probably check to see if it is getting good results over time, rather than spot checking. Probably good to spot check at first though.

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
vs = analyzer.polarity_scores(tweet)
print(vs["compound"], vs["pos"], vs["neu"], vs["neg"])


## if You don't get good results with Vader. Then use the data you've stored to build and test your own model. 
#IF, and that's a big if, it performs better than VADER, then use it. Bonus points for building your own shit, I mean the longest part is gonna be collecting enough usuable data. 

# Historical Data
Unfortunately, Tweepy's library only supports basic search functions, making it suitable for 7-day - real-time updates. To look at historical data, we can use Twitter's own in-house library, Twitter (see https://twitterdev.github.io/search-tweets-python/ for documentation.) The majority of the retrospective analysis will be done doing this. Key question we seek to answer: - What were the trends that led to GDPR & CCPA being adopted? 